<style>
.brown {
    color: #9c6644;
    font-weight:500;
    font-size: 40px;
    text-align: button;
    padding: 0;
    margin: 0;
}
</style>
<style>
.light-brown {
    color: #a78a7f;
    font-weight:500;
    font-size: 25px;
    text-align: top;
    padding: 0;
    margin: 0;
}
</style>
<style>
.linea {
  border-top: 2px solid #9c6644;
  height: 2px;
  max-width: 535px;
  padding: 10;
  margin: 10;
}
</style>
<p class="brown">
    PROJECT MLOps STREAMING
</p>

<div class="linea"></div>

<p class="light-brown">
    DATASET LOADING AND TRANSFORMATION (PLATFORM)
</p>

**GOALS**

- Generate a column id: <br>
    With the first letter of the platform, plus the show_id field.


- Replace null values ​​in the Rating column with the letter 'G'


- Dates must be in the format YYYY-mm-dd


- Text fields must be in lower case, no exceptions


- The duration field must be converted into two fields;<br>
    a). duration_int: Will be an integer.<br>
    b). duration_type: String indicating the unit of duration measurement.

In [1]:
# Import the libraries.
import numpy as np  
import pandas as pd
import pyarrow

# Load the datasets.
dfAmazonPrime = pd.read_csv('./datasets/amazon_prime_titles.csv')
dfDisneyPlus = pd.read_csv('./datasets/disney_plus_titles.csv')
dfHulu = pd.read_csv('./datasets/hulu_titles.csv')
dfNetflix = pd.read_csv('./datasets/netflix_titles.csv')

# Visualize the dataset.
print('dfAmazonPrime: rows', dfAmazonPrime.shape, 'columns.')
print('dfDisneyPlus:  rows', dfDisneyPlus.shape, 'columns.')
print('dfHulu:        rows', dfHulu.shape, 'columns.')
print('dfNetflix:     rows', dfNetflix.shape, 'columns.')
print('')

# Create a function to check that they bring the same columns
def compare_datasets(dataset1, dataset2):
    if set(dataset1) != set(dataset2):
        return print('The data sets consulted are different.',set(dataset1), set(dataset2))
    return print('The queried datasets bring the same columns.')
compare_datasets(dfAmazonPrime, dfDisneyPlus)
compare_datasets(dfAmazonPrime, dfHulu)
compare_datasets(dfAmazonPrime, dfNetflix)

dfAmazonPrime: rows (9668, 12) columns.
dfDisneyPlus:  rows (1450, 12) columns.
dfHulu:        rows (3073, 12) columns.
dfNetflix:     rows (8807, 12) columns.

The queried datasets bring the same columns.
The queried datasets bring the same columns.
The queried datasets bring the same columns.


In [2]:
# Generate a new column id.
dfAmazonPrime.insert(loc=0, column='id', value='a' + dfAmazonPrime['show_id'])
dfDisneyPlus.insert(loc=0, column='id', value='d' + dfDisneyPlus['show_id'])
dfHulu.insert(loc=0, column='id', value='n' + dfHulu['show_id'])
dfNetflix.insert(loc=0, column='id', value='n' + dfNetflix['show_id'])

# And I create another one with the name of the platform.
dfAmazonPrime.insert(loc=1, column='platform', value='amazon prime')
dfDisneyPlus.insert(loc=1, column='platform', value='disney plus')
dfHulu.insert(loc=1, column='platform', value='hulu')
dfNetflix.insert(loc=1, column='platform', value='netflix')

# Concatenate the datasets.
dfAllPlatform = pd.concat([dfAmazonPrime, 
                           dfDisneyPlus, 
                           dfHulu, 
                           dfNetflix])

# Show the result.
print('dfAllPlatform: rows', dfAllPlatform.shape, 'columns.')
print('')
print('*'*40)
print('')
print(dfAllPlatform.isnull().sum())
print('')
print('*'*40)

dfAllPlatform: rows (22998, 14) columns.

****************************************

id                  0
platform            0
show_id             0
type                0
title               0
director         8259
cast             5321
country         11499
date_added       9554
release_year        0
rating            864
duration          482
listed_in           0
description         4
dtype: int64

****************************************


In [3]:
# Check the data I have in the Rating column.
check = list(dfAllPlatform['rating'].unique())
print('The rate data is:', check)
dfAllPlatform.head(1)

The rate data is: [nan, '13+', 'ALL', '18+', 'R', 'TV-Y', 'TV-Y7', 'NR', '16+', 'TV-PG', '7+', 'TV-14', 'TV-NR', 'TV-G', 'PG-13', 'TV-MA', 'G', 'PG', 'NC-17', 'UNRATED', '16', 'AGES_16_', 'AGES_18_', 'ALL_AGES', 'NOT_RATE', 'TV-Y7-FV', 'NOT RATED', '2 Seasons', '93 min', '4 Seasons', '136 min', '91 min', '85 min', '98 min', '89 min', '94 min', '86 min', '3 Seasons', '121 min', '88 min', '101 min', '1 Season', '83 min', '100 min', '95 min', '92 min', '96 min', '109 min', '99 min', '75 min', '87 min', '67 min', '104 min', '107 min', '84 min', '103 min', '105 min', '119 min', '114 min', '82 min', '90 min', '130 min', '110 min', '80 min', '6 Seasons', '97 min', '111 min', '81 min', '49 min', '45 min', '41 min', '73 min', '40 min', '36 min', '39 min', '34 min', '47 min', '65 min', '37 min', '78 min', '102 min', '129 min', '115 min', '112 min', '61 min', '106 min', '76 min', '77 min', '79 min', '157 min', '28 min', '64 min', '7 min', '5 min', '6 min', '127 min', '142 min', '108 min', '57 min

,id,platform,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,as1,amazon prime,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...


In [4]:
# Replace null values ​​in the Rating column with the letter 'G'.
dfAllPlatform['rating'] = dfAllPlatform['rating'].replace(np.nan, 'G')

# And the values ​​not belonging to the rating column I take them to their respective column.
# For that, I create two columns temporarily.
dfAllPlatform['mins'] = dfAllPlatform['rating']
dfAllPlatform['seasons'] = dfAllPlatform['rating']

# Create two boolean masks that are True for values ​​ending in 'min' o contains 'Season'.
maskMins = dfAllPlatform['mins'].str.endswith('min')
maskSeasons = dfAllPlatform['seasons'].str.contains('Season')

# I assign NaN to column values ​​that do not satisfy the boolean mask.
dfAllPlatform.loc[~maskMins, 'mins'] = np.nan
dfAllPlatform.loc[~maskSeasons, 'seasons'] = np.nan

# I combine the columns into a single one.
dfAllPlatform['duration'] = dfAllPlatform['duration'].combine_first(dfAllPlatform['mins'])
dfAllPlatform['duration'] = dfAllPlatform['duration'].combine_first(dfAllPlatform['seasons'])

# Replace the values ​​that did not belong to the rating column.
dfAllPlatform.loc[maskMins, 'rating'] = 'G'
dfAllPlatform.loc[maskSeasons, 'rating'] = 'G'

# Check the data I have in the Rating column.
check = list(dfAllPlatform['rating'].unique())
print('The rate data is:', check)
dfAllPlatform.head(1)

The rate data is: ['G', '13+', 'ALL', '18+', 'R', 'TV-Y', 'TV-Y7', 'NR', '16+', 'TV-PG', '7+', 'TV-14', 'TV-NR', 'TV-G', 'PG-13', 'TV-MA', 'PG', 'NC-17', 'UNRATED', '16', 'AGES_16_', 'AGES_18_', 'ALL_AGES', 'NOT_RATE', 'TV-Y7-FV', 'NOT RATED', 'UR']


,id,platform,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,mins,seasons
0,as1,amazon prime,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,G,113 min,"Comedy, Drama",A small fishing village must procure a local d...,NaN,NaN


In [5]:
# I replace the format of the date in the requested one.
dfAllPlatform['date_added'] = pd.to_datetime(dfAllPlatform['date_added']).dt.strftime('%Y-%m-%d')

# Convert all letters in the dataset to lowercase.
dfAllPlatform = dfAllPlatform.apply(lambda x: x.str.lower() if x.dtypes == 'object' else x)

# Split the "duration" column into duration_int and duration_type.
dfAllPlatform[['duration_int', 'duration_type']] = dfAllPlatform['duration'].str.extract('(\d+)\s*(\w+)')

# And in the duration_type column I replaced 'seasons' with 'season'.
dfAllPlatform['duration_type'] = dfAllPlatform['duration_type'].str.replace('seasons', 'season')

# Viewed the dataset.
dfAllPlatform.head(5)

,id,platform,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,mins,seasons,duration_int,duration_type
0,as1,amazon prime,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113 min,"comedy, drama",a small fishing village must procure a local d...,NaN,NaN,113,min
1,as2,amazon prime,s2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110 min,"drama, international",a metro family decides to fight a cyber crimin...,NaN,NaN,110,min
2,as3,amazon prime,s3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,74 min,"action, drama, suspense",after a man discovers his wife is cheating on ...,NaN,NaN,74,min
3,as4,amazon prime,s4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,69 min,documentary,"pink breaks the mold once again, bringing her ...",NaN,NaN,69,min
4,as5,amazon prime,s5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,45 min,"drama, fantasy",teenage matt banting wants to work with a famo...,NaN,NaN,45,min


<style>
.linea {
  border-top: 2px solid #a78a7f;
  height: 2px;
  max-width: 2000px;
  padding: 10;
  margin: 10;
}
</style>
<div class="linea"></div>
<div class="linea"></div>


<style>
.light-brown {
    color: #a78a7f;
    font-weight:500;
    font-size: 25px;
    text-align: top;
    padding: 0;
    margin: 0;
}
</style>
<p class="light-brown">
    DATASET LOADING AND TRANSFORMATION (RATING)
</p>

**GOALS**

- I rename the columns: <br>
    a). 'rating' for 'score'<br>
    b). 'movieid' for 'id'.

- Dates must be in the format YYYY-mm-dd

- Text fields must be in lower case, no exceptions

- Generated an average column.


In [6]:
# Load the datasets.
dfRating1= pd.read_csv('./datasets/ratings/1.csv')
dfRating2= pd.read_csv('./datasets/ratings/2.csv')
dfRating3= pd.read_csv('./datasets/ratings/3.csv')
dfRating4= pd.read_csv('./datasets/ratings/4.csv')
dfRating5= pd.read_csv('./datasets/ratings/5.csv')
dfRating6= pd.read_csv('./datasets/ratings/6.csv')
dfRating7= pd.read_csv('./datasets/ratings/7.csv')
dfRating8= pd.read_csv('./datasets/ratings/8.csv')

# Visualize the dataset.
print('dfRating1: rows', dfRating1.shape, 'columns.')
print('dfRating2: rows', dfRating2.shape, 'columns.')
print('dfRating3: rows', dfRating3.shape, 'columns.')
print('dfRating4: rows', dfRating4.shape, 'columns.')
print('dfRating5: rows', dfRating5.shape, 'columns.')
print('dfRating6: rows', dfRating6.shape, 'columns.')
print('dfRating7: rows', dfRating7.shape, 'columns.')
print('dfRating8: rows', dfRating8.shape, 'columns.')
print('')
# Check with the function above, if it brings the same columns to the dataset.
compare_datasets(dfRating1, dfRating2)
compare_datasets(dfRating1, dfRating3)
compare_datasets(dfRating1, dfRating4)
compare_datasets(dfRating1, dfRating5)
compare_datasets(dfRating1, dfRating6)
compare_datasets(dfRating1, dfRating7)
compare_datasets(dfRating1, dfRating8)

# Query a .head() of one of the datasets
dfRating8.head(1)

dfRating1: rows (1500000, 4) columns.
dfRating2: rows (1500000, 4) columns.
dfRating3: rows (1500000, 4) columns.
dfRating4: rows (1500000, 4) columns.
dfRating5: rows (1500000, 4) columns.
dfRating6: rows (1500000, 4) columns.
dfRating7: rows (524289, 4) columns.
dfRating8: rows (1500000, 4) columns.

The queried datasets bring the same columns.
The queried datasets bring the same columns.
The queried datasets bring the same columns.
The queried datasets bring the same columns.
The queried datasets bring the same columns.
The queried datasets bring the same columns.
The queried datasets bring the same columns.


,userId,rating,timestamp,movieId
0,108422,4.5,1350689762,as1399


In [7]:
# I concatenate all the datasets into one, to apply the requested changes to just one...
dfAllRating = pd.concat([dfRating1, 
                         dfRating2, 
                         dfRating3, 
                         dfRating4, 
                         dfRating5, 
                         dfRating6, 
                         dfRating7, 
                         dfRating8])

# I rename the requested columns.
dfAllRating = dfAllRating.rename(columns={'rating':'score',
                                          'movieId':'id'})

# Changed the date to the specified format.
dfAllRating['timestamp'] = pd.to_datetime(dfAllRating['timestamp'], unit='s')
dfAllRating['timestamp'] = dfAllRating['timestamp'].dt.strftime('%Y-%m-%d')

# Convierto todas las letras del dataset en minusculas.
dfAllRating = dfAllRating.apply(lambda x: x.str.lower() if x.dtypes == 'object' else x)

# I see how the dataset is turning out.
print('dfAllRating: rows', dfAllRating.shape, 'columns.')
dfAllRating.head(10)


dfAllRating: rows (11024289, 4) columns.


,userId,score,timestamp,id
0,1,1.0,2015-03-09,as680
1,1,4.5,2015-03-09,ns2186
2,1,5.0,2015-03-09,hs2381
3,1,5.0,2015-03-09,ns3663
4,1,5.0,2015-03-09,as9500
5,1,4.0,2015-03-09,as3004
6,1,4.5,2015-03-09,ns8282
7,1,5.0,2015-03-09,as6112
8,1,4.0,2015-03-09,hs327
9,1,4.0,2015-03-09,ns1358


In [8]:
# Create a dataset with the average of the movies grouped by id.
dfAverage = dfAllRating.groupby('id')['score'].mean().round(2)
dfAverage = pd.DataFrame({'id': dfAverage.index, 
                          'average': dfAverage.values})
dfAverage

,id,average
0,as1,3.47
1,as10,3.44
2,as100,3.61
3,as1000,3.56
4,as1001,3.59
...,...,...
22993,ns995,3.52
22994,ns996,3.63
22995,ns997,3.53
22996,ns998,3.58


<style>
.linea {
  border-top: 2px solid #a78a7f;
  height: 2px;
  max-width: 2000px;
  padding: 10;
  margin: 10;
}
</style>
<div class="linea"></div>
<div class="linea"></div>


<style>
.light-brown {
    color: #a78a7f;
    font-weight:500;
    font-size: 25px;
    text-align: top;
    padding: 0;
    margin: 0;
}
</style>
<p class="light-brown">
    MERGE THE DATASET
</p>

In [9]:
# DATASET that i'm going to use for the FastApi.
dfStreamingFA = pd.merge(dfAllPlatform, dfAverage, on='id')
print('dfStreamingFA: rows', dfStreamingFA.shape, 'columns.')
dfStreamingFA.head(1)

dfStreamingFA: rows (22998, 19) columns.


,id,platform,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,mins,seasons,duration_int,duration_type,average
0,as1,amazon prime,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113 min,"comedy, drama",a small fishing village must procure a local d...,NaN,NaN,113,min,3.47


In [10]:
# DATASET that i'm going to use for the Machine Learning Model.
dfStreamingML = pd.merge(dfAllRating, dfAllPlatform, on='id')
print('dfStreamingML: rows', dfStreamingML.shape, 'columns.')
dfStreamingML.head(1)

dfStreamingML: rows (11024324, 21) columns.


,userId,score,timestamp,id,platform,show_id,type,title,director,cast,...,date_added,release_year,rating,duration,listed_in,description,mins,seasons,duration_int,duration_type
0,1,1.0,2015-03-09,as680,amazon prime,s680,tv show,the english civil war,NaN,robert whelan,...,NaN,2019,13+,1 season,"documentary, special interest",a nation divided. it was a time of great bitte...,NaN,NaN,1,season


<style>
.linea {
  border-top: 2px solid #a78a7f;
  height: 2px;
  max-width: 2000px;
  padding: 10;
  margin: 10;
}
</style>
<div class="linea"></div>
<div class="linea"></div>

<style>
.light-brown {
    color: #a78a7f;
    font-weight:500;
    font-size: 25px;
    text-align: top;
    padding: 0;
    margin: 0;
}
</style>
<p class="light-brown">
    CLEAN THE DATA...
</p>

**1. First I work with the API dataset.**

In [11]:
# Display the columns I have... 
# with the sum of the null values.
print('*'*40)
print('')
print(dfStreamingFA.isnull().sum())
print('')
print('*'*40)

****************************************

id                   0
platform             0
show_id              0
type                 0
title                0
director          8259
cast              5321
country          11499
date_added        9554
release_year         0
rating               0
duration           187
listed_in            0
description          4
mins             22765
seasons          22936
duration_int       187
duration_type      187
average              0
dtype: int64

****************************************


In [12]:
# I remove the columns that I won't need for the DATASET of the API.
dfStreamingFA.drop(['id', 
                    'show_id', 
                    'type', 
                    'director', 
                    'country', 
                    'date_added', 
                    'rating', 
                    'duration', 
                    'listed_in', 
                    'description', 
                    'mins', 
                    'seasons', 
                    ],axis=1, inplace=True)

# Replace null values ​​with the number 0, or with the text string 'no data'.
dfStreamingFA['duration_int'] = dfStreamingFA['duration_int'].fillna('0')
dfStreamingFA = dfStreamingFA.fillna('no data')

# Check the changes with (info), to obtain the data type of each column...
print('*'*40)
print('')
print(dfStreamingFA.info())
print('')
print('*'*40)

****************************************

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22998 entries, 0 to 22997
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   platform       22998 non-null  object 
 1   title          22998 non-null  object 
 2   cast           22998 non-null  object 
 3   release_year   22998 non-null  int64  
 4   duration_int   22998 non-null  object 
 5   duration_type  22998 non-null  object 
 6   average        22998 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 1.4+ MB
None

****************************************


In [13]:
# The only column, that I have to replace the data type is the one from duration_int to: int64.
dfStreamingFA['duration_int'] = dfStreamingFA['duration_int'].astype('int64')

**2. Second job with the Machine Learning dataset...**

In [14]:
# Display the columns I have... 
# with the sum of the null values ​​and the data type.
print('*'*40)
print('')
print(dfStreamingML.isnull().sum())
print('')
print('*'*40)

****************************************

userId                  0
score                   0
timestamp               0
id                      0
platform                0
show_id                 0
type                    0
title                   0
director          3960633
cast              2550894
country           5509941
date_added        4577425
release_year            0
rating                  0
duration            89325
listed_in               0
description          1986
mins             10912666
seasons          10994544
duration_int        89325
duration_type       89325
dtype: int64

****************************************


In [15]:
# I remove the columns that I will not need for the DATASET of the ML Model.
dfStreamingML.drop(['timestamp', 
                    'show_id', 
                    'type', 
                    'duration', 
                    'description', 
                    'mins', 
                    'seasons', 
                    ],axis=1, inplace=True)

# Replace null values ​​with the number 0, or with the text string 'no data'.
dfStreamingML['duration_int'] = dfStreamingML['duration_int'].fillna('0')
dfStreamingML[['director', 'cast', 'country']] = dfStreamingML[['director', 'cast', 'country']].fillna('no data')

# Check the changes with (info), to obtain the data type of each column...
print('*'*40)
print('')
print(dfStreamingML.info())
print('')
print('*'*40)
dfStreamingML.head(1)

****************************************

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11024324 entries, 0 to 11024323
Data columns (total 14 columns):
 #   Column         Dtype  
---  ------         -----  
 0   userId         int64  
 1   score          float64
 2   id             object 
 3   platform       object 
 4   title          object 
 5   director       object 
 6   cast           object 
 7   country        object 
 8   date_added     object 
 9   release_year   int64  
 10  rating         object 
 11  listed_in      object 
 12  duration_int   object 
 13  duration_type  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 1.2+ GB
None

****************************************


,userId,score,id,platform,title,director,cast,country,date_added,release_year,rating,listed_in,duration_int,duration_type
0,1,1.0,as680,amazon prime,the english civil war,no data,robert whelan,no data,NaN,2019,13+,"documentary, special interest",1,season


In [16]:
# Replace the data type in the columns, duration_int, release_year and date_added...
dfStreamingML['duration_int'] = dfStreamingML['duration_int'].astype('int64')
dfStreamingML['release_year'] = pd.to_datetime(dfStreamingML['release_year'], format='%Y-%m-%d').dt.year
dfStreamingML['date_added'] = pd.to_datetime(dfStreamingML['date_added'], format='%Y-%m-%d')

# Check the unique values ​​of the rating column again.
check = list(dfStreamingML['rating'].unique())
print('The rate data is:', check)

The rate data is: ['13+', 'g', 'tv-y', 'tv-14', 'pg-13', 'tv-nr', 'tv-ma', 'tv-pg', 'pg', 'tv-g', '16+', '7+', 'nr', 'tv-y7', 'all', 'r', '18+', 'tv-y7-fv', 'ur', 'unrated', 'nc-17', 'ages_18_', '16', 'not rated', 'not_rate', 'ages_16_', 'all_ages']


In [17]:
# As a result, I see that I have several types of rating that indicate being the same with different names, therefore I seek to standardize all the values...

'''
TV-Y:       Suitable for all audiences
TV-Y7:      Suitable for children over 7 years old
TV-Y7-FV:   Suitable for children over 7 years of age with fictional violence
TV-G:       Suitable for all ages
TV-PG:      Parental Guidance Suggested
PG:         Parental Guidance Suggested
PG-13:      Not recommended for children under 13 without parental supervision
TV-14:      Not recommended for children under 14 without parental supervision
A:          Restricted - Children under 17 years of age require the accompaniment of a parent or responsible adult
NC-17:      Only for adults over 18 years of age
TV-MA:      Only for adults
NR:         Not Rated
'''

# Create a dictionary to map the data I have in the column to the standard value set...
standarRating = {
    '13+':          'pg-13',
    'g':            'tv-g',
    'tv-y':         'tv-y',
    'tv-14':        'tv-14',
    'pg-13':        'pg-13',
    'tv-nr':        'tv-nr',
    'tv-ma':        'tv-ma',
    'tv-pg':        'tv-pg',
    'pg':           'tv-pg',
    'tv-g':         'tv-g',
    '16+':          'tv-16',
    '7+':           'tv-y7',
    'nr':           'tv-nr',
    'tv-y7':        'tv-y7',
    'all':          'tv-y',
    'r':            'r',
    '18+':          'nc-17',
    'tv-y7-fv':     'tv-y7-fv',
    'ur':           'tv-nr',
    'unrated':      'tv-nr',
    'nc-17':        'nc-17',
    'ages_18_':     'nc-17',
    '16':           'tv-16',
    'not rated':    'tv-nr',
    'not_rate':     'tv-nr',
    'ages_16_':     'tv-16',
    'all_ages':     'tv-g'
}

# Replace the values ​​of my columns with the mapping that I did in the previous step.
dfStreamingML['rating'] = dfStreamingML['rating'].map(standarRating)

# Check the unique values ​​of the rating column again.
check = list(dfStreamingML['rating'].unique())
print('The rate data is:', check)

The rate data is: ['pg-13', 'tv-g', 'tv-y', 'tv-14', 'tv-nr', 'tv-ma', 'tv-pg', 'tv-16', 'tv-y7', 'r', 'nc-17', 'tv-y7-fv']


<style>
.linea {
  border-top: 2px solid #a78a7f;
  height: 2px;
  max-width: 2000px;
  padding: 10;
  margin: 10;
}
</style>
<div class="linea"></div>
<div class="linea"></div>

<style>
.light-brown {
    color: #a78a7f;
    font-weight:500;
    font-size: 25px;
    text-align: top;
    padding: 0;
    margin: 0;
}
</style>
<p class="light-brown">
    EXPORT THE WORKED DATASET.
</p>

In [18]:
# Export the file to parquet.
dfStreamingFA.to_parquet('StreamingFA.parquet', index=False)

In [19]:
# Export the file to parquet.
dfStreamingML.to_parquet('StreamingML.parquet', index=False)

<style>
.light-brown {
    color: #463f3a;
    font-weight:500;
    font-size: 20px;
    vertical-align: middle;
    text-align: right;
    padding: 0px;
    margin: 0px 0px 20px 0px;
}
</style>
<style>
.linea {
  border-top: 0.5px solid #463f3a;
  height: 1px;
  max-width: 2000px;
  padding: 0;
  margin: 0;
}
</style>
<div class="linea"></div>
<p class="light-brown">
    Project carried out by Daniele, Emiliano Gastón.<br>
    Thanks for your time!
</p>
<div class="linea"></div>
<div class="linea"></div>